In [10]:
import numpy as np
import pandas as pd
import pickle


**Part 1. Generate Raw Data (i.e., Percept Cues for the stories from BigToM and OpenToM)**

**BigToM**

1. First, we preview the BigToM dataset

In [231]:
new_bigtom = pd.read_csv("bigtom-modified.csv", delimiter = ';')
new_bigtom
# we can edit on top of this

,Story - Context,Story - Desire,Story - Perception cues,Story - Belief,Story - An external event,Percepts - Aware of event,Percepts - Not aware of event,Actions - Action given new state,Actions - Action given initial state,Questions - Belief Question,...,Answers - Belief Aware,Answers - Desire Aware,Answers - Action Aware,Answers - Belief not Aware,Answers - Desire not Aware,Answers - Action not Aware,Random Event,Percepts of R.E. - Aware of random event,Percepts of R.E. - not Aware of random event,21
0,Noor is working as a barista at a busy coffee ...,Noor wants to make a delicious cappuccino for...,Noor grabs a milk pitcher and fills it with o...,Noor believes that the milk pitcher contains ...,"A coworker, who didn't hear the customer's re...",Noor sees her coworker swapping the milk.,Noor does not see her coworker swapping the milk.,Noor opens the fridge again and reaches for th...,Noor makes the cappuccino using the milk in th...,Does Noor believe the milk pitcher contains oa...,...,Noor believes the milk pitcher contains almond...,Noor wants to make a cappuccino with oat milk.,Noor will open the fridge once again to take o...,Noor believes the milk pitcher contains oat milk.,Noor wants to make a cappuccino with oat milk.,Noor will make the cappuccino using the milk i...,A decorative mug falls from a shelf in the cof...,Noor sees the decorative mug falling from the ...,Noor does not see the decorative mug falling f...,auto;0
1,Zara is attending an art exhibition at a local...,Zara wants to learn more about a particular a...,The label next to the artwork states that the...,Zara believes that the painting is the work o...,"A gallery staff member, realizing there was a...",Zara reads the updated label carefully.,Zara does not read the updated label.,Zara looks for more information about the emer...,Zara looks for more information about the famo...,Does Zara believe the painting is the work of ...,...,Zara believes the painting is the work of an e...,Zara wants to learn more about the painting an...,Zara will look for more information about the ...,Zara believes the painting is the work of a fa...,Zara wants to learn more about the painting an...,Zara will look for more information about the ...,A gallery visitor accidentally spills a glass ...,Zara observes the water spill.,Zara does not observe the water spill.,auto;0
2,Carlos is a farmer in a small village in Mexic...,He wants to irrigate his crops by opening a w...,Carlos closed the valve the previous evening.,Carlos believes the valve is currently closed.,"During the night, heavy rainfall caused the r...",Carlos sees the flooded cornfield.,Carlos does not see the flooded cornfield.,Carlos calls his neighbour to ask them if they...,Carlos calls his neighbour to ask them if they...,Does Carlos believe the water valve is closed ...,...,Carlos believes the water valve is open.,Carlos wants to close the valve due to the flo...,Carlos will call his neighbour to ask them if ...,Carlos believes the water valve is closed.,Carlos wants to open the valve to irrigate his...,Carlos will call his neighbour to ask them if ...,A group of wild turkeys wanders into the cornf...,Carlos notices the wild turkeys in his field.,Carlos does not notice the wild turkeys in his...,auto;0
3,Kofi is a fisherman from a small village in Gh...,He wants to catch enough fish today to provid...,Kofi repaired his fishing net last night.,Kofi believes that his fishing net is in good...,"While Kofi is away from his boat, a group of ...",Kofi observes the monkeys damaging his fishing...,Kofi does not see the monkeys damaging his fis...,Kofi buys new repair equipment for fixing the ...,Kofi goes fishing with his net.,Does Kofi believe his fishing net is in good c...,...,Kofi believes his fishing net is torn apart.,Kofi wants to catch fish with his net in good ...,Kofi will buy new repair equipment for fixing ...,Kofi believes his fishing net is in good condi...,Kofi wants to catch fish with his net in good ...

2. Load essential libraries and define essential functions.

In [ ]:
import json
import re
import sys
import time
import warnings
from random import sample

import openai
from openai import AzureOpenAI
import pandas as pd
# from tqdm import tqdm
import tiktoken
key = "insert_key_here" # insert key

# Parameters
client = AzureOpenAI(
  azure_endpoint = "https://hkust.azure-api.net",
  api_version = "2024-06-01",
  api_key = key #put your api key here
)

# Function
def get_response(message, instruction):
    response = client.chat.completions.create(
		model = 'gpt-4',
        temperature = 1,
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": message}
        ]
    )
    
    # print token usage
    print(response.usage)
    # return the response
    return response.choices[0].message.content
  
def LM_generation(prompt, max_tokens = 10, temperature = 0.1, model = "gpt-4", system_msg = None, msg_history = None):
    retry_attempt = 5
    retry_num = 0
    generation_success = False
    messages = []
    if system_msg is not None:
        messages.append({'role':'system', 'content': system_msg})
    if msg_history is not None:
        messages = msg_history
    messages.append({'role':'user', 'content':prompt})
    while retry_num < retry_attempt and not generation_success:
        try:
            gen = client.chat.completions.create(
                model = model,
                messages = messages,
                max_tokens = max_tokens,
                temperature = temperature,
            ) 
            generation_success = True
            input_tokens = gen.usage.prompt_tokens
            output_tokens = gen.usage.completion_tokens
        except openai.APIError as e:
            retry_num += 1
            generation_success = False
            time.sleep(5)
            # return "openai.error.APIError", -1, -1 
        except openai.RateLimitError as e:
            retry_num += 1
            generation_success = False
            time.sleep(30)
        except:
            retry_num += 1
            generation_success = False
            time.sleep(10)
    
    if generation_success:
        if gen == None or gen.choices[0] == None or gen.choices[0].message == None or gen.choices[0].message.content == None:
            return "", 0, 0
        return gen.choices[0].message.content.strip(), input_tokens, output_tokens
    else:
        return "Error", -1,-1

def count_tokens(text, encoding_name = "cl100k_base"):
    encoding = tiktoken.get_encoding(encoding_name)
    # print(text)
    num_tokens = len(encoding.encode(text))
    return num_tokens


3. Create a list of names of protagonists.

In [232]:
protag_name = [new_bigtom.iloc[i]['Story - Context'].split()[0] for i in range(new_bigtom.shape[0])]
print(protag_name)

['Noor', 'Zara', 'Carlos', 'Kofi', 'Azola', 'Ravi', 'Kofi', 'Kael', 'Farhan', 'Abhay', 'Aniket', 'Lila', 'Kavi', 'Hiroshi', 'Juan', 'Mei', 'Juanita', 'Mei', 'Cheng', 'Jia', 'Isabela', 'Emeka', 'Luka', 'Amaury', 'Chi', 'Mei', 'Mei', 'Nia', 'Heiwa', 'Amina', 'Fatima', 'Fatima', 'Nneka', 'Fatima', 'Amina', 'Aiko', 'Carlos', 'Mateo', 'Miguel', 'Carlos', 'Carlos', 'Amara', 'Fatima', 'Fatima', 'Aisha', 'Amara', 'Juan', 'Hiroshi', 'Yoshi,', 'Takumi', 'Hiro', 'Tumelo', 'Kofi', 'Lila', 'Odette', 'Mei', 'Fatima', 'Kwame', 'Luka', 'Olumide,', 'Odion', 'Ibrahim', 'Rohan', 'Rohit', 'Kunal', 'Amara', 'Leilani', 'Amina', 'Amara,', 'Amina', 'Fatima', 'Mei', 'Yara', 'Isabella', 'Xia', 'Mei', 'Olabisi', 'Kalyani', 'Kavya', 'Amara', 'Naledi', 'Emilia', 'Mei', 'Emeka', 'Aria', 'Ailani', 'Takeshi', 'Ravi', 'Surya', 'Takeshi', 'Hiroshi', 'Kwame', 'Ngozi', 'Naledi', 'Lila', 'Amara', 'Mei,', 'Mei,', 'Jia', 'Enrique', 'Amara', 'Kwasi', 'Fatima', 'Kofi', 'Kofi', 'Amrita', 'Amina', 'Priya', 'Nkiru', 'Aisha', 'Ta

4. Generate Percept Cues

In [ ]:

# process story
# given string input, return structuralized story 

def process(percept_text, senses, mode = 'default'):
    # given string input of story
    percept_text = percept_text.split('\n')  # format for each cue: 'cue name', 'cue 1', 'cue 2', blank.
    if mode == 'default':
        if len(percept_text) not in [4*len(senses)-1, 4*len(senses)]:
            print('process() not done correctly')
        return {senses[i]:[percept_text[4*i+1][3:], percept_text[4*i+2][3:]] for i in range(len(senses))}
    
    elif mode == 'random':
        if len(percept_text) not in [3*len(senses)-1, 3*len(senses)]:
            print('process() not done correctly')
        return {senses[i]:percept_text[3*i+1] for i in range(len(senses))}
    
    elif mode == 'opentom_default':
        print(len(percept_text))
        if len(percept_text) not in [3*len(senses)-1, 3*len(senses)]:
            print('process() not done correctly')
        return {senses[i]:[percept_text[3*i+1][3:], percept_text[3*i+2][3:]] for i in range(len(senses))}
    elif mode == 'opentom_random':
        if len(percept_text) not in [2*len(senses)-1, 2*len(senses)]:
            print('process() not done correctly')
        return {senses[i]:percept_text[2*i+1] for i in range(len(senses))}
    

# senses:
senses = ['visual', 'auditory', 'olfactory', 'gustatory', 'tactile', 'proprio'] # proprio = proprioception
num_senses = len(senses)

In [233]:
num_stories = new_bigtom.shape[0] # num_stories in bigtom
print(num_stories)

200


In [24]:
#save

def save_list_of_dict(save_name, list_of_dict):
    with open(save_name, 'wb') as f:
        pickle.dump(list_of_dict, f)

def load_list_of_dict(save_name):
    with open(save_name, 'rb') as f:
        loaded = pickle.load(f)
    return loaded

We plan to only generate 100 stories, using GPT-4o-mini. You may also use Deepseek V3.

In [ ]:
num_bigtom_stories = 100

You may want to change num_iters to 2 or 3 if you wish to generate more percepts. This is what I did.

In [ ]:
# Generate Story Template
# i = 1
import time 
input_tokens = 0
output_tokens = 0

begin = time.time()

percept_processed = []

for i in range(num_bigtom_stories): 
    print("Generating:", i, "-th prompt")
    prompt = f"Story: {new_bigtom.iloc[i]['Story - Context']} {new_bigtom.iloc[i]['Story - Desire']} {new_bigtom.iloc[i]['Story - Perception cues']}\n\n"
    prompt += f"Main character's belief: {new_bigtom.iloc[i]['Story - Belief']}\n\n"
    # prompt = f"Story: {new_bigtom.iloc[i]['Story - Context']} {new_bigtom.iloc[i]['Story - Desire']} {new_bigtom.iloc[i]['Story - Perception cues']} \n\n"
    prompt += f"Event: {new_bigtom.iloc[i]['Story - An external event']} \n\n"
    
    # prompt += f"Instructions: From this Story, generate 10 perceptual cues that {protag_name[i]} may notice about the Event. "
    # prompt += f"Each perceptual cue should continue the Story and Event. Write one sentence for each perceptual cue. "
    # prompt += f"Avoid descriptions of {protag_name[i]}\'s thoughts. Be systematic, reasonable, creative, and innovative, and make the stories diverse (from different contexts). "
    # prompt += f"Avoid making direct comparisons to the item mentioned in the event, but make vivid descriptions about the difference."
    
    # 
    #  visual, auditory, olfactory, gustatory, tactile, 
    prompt += "Instructions: Based on the following story and event, generate perceptual cues that the main character might encounter, "
    prompt += "which MUST challenge their belief. Each cue should be creative, reasonable, and systematically explore the sensory modalities of: visual, auditory, olfactory, gustatory, tactile, proprioception (sense of body position/weight). "
    prompt += "Be descriptive and only make vivid descriptions about the perceptual cue, without describing the main character's thoughts. Avoid making explicit comparisons between the event and the main character's belief. Generate 2 perceptual cues for each sensory modality. "
    prompt += "Be super duper creative! "
    prompt += "Do not use these words: different, difference, instead of, contrasts.\n"
    prompt += "For each cue, write your output in a format like this example: \n"
    prompt += "### Visual Cues\n1. A bright green label on the milk carton nearby suddenly catches Noor’s eye, glistening under the café's overhead lights. The bold print reads “Almond Milk” in a slightly whimsical font, contrasting sharply with the soft beige of the oat milk packaging Noor is accustomed to seeing.\n 2. As Noor pours the milk into the espresso, she notices the liquid's texture. Instead of the familiar creamy, smooth flow that oat milk typically has, the milk appears slightly more translucent, with a thinner consistency that swirls differently in the cup, creating delicate, lacy patterns.\n\n"

    # prompt += 'Write your output in this template: “### <sense 1> Cues\n1. <perceptual cue (variable length)>\n2. <perceptual cue (variable length)>\n\n### <sense 2> Cues\n1. <perceptual cue (variable length)>\n2. <perceptual cue (variable length)>”'
    # prompt += "Here are some general examples of how perceptual cues should look like, for your reference: "
    # prompt += "1. Maria sees smoke coming from the kitchen, suggesting a fire. Should she change her belief about the kitchen being safe? "
    # prompt += "2. Lila smells gas in the garage, where she believes everything is secure. Should she change her belief about the safety of the garage? "
    
    # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
    input_tokens += count_tokens(prompt)
    
    # print(prompt) # if you wish to print outputs. 
    num_iters = 1
    for _ in range(num_iters): # run more times if you wish to generate more percept cues. 
        # print("iteration ")
        max_tokens = 1500
        output = LM_generation(prompt, temperature = 0.6, max_tokens = max_tokens, model = "gpt-4o-mini")
        # print(output)
        # put output into the percept_processed
        percept_processed.append(process(output[0], senses))
        output_tokens += count_tokens(output[0])
        if (count_tokens(output[0])>1200):
            print("Warning: long input")
        if (count_tokens(output[0])>max_tokens):
            print("Error: LLM didn't manage to finish. Plz check again")
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
        save_list_of_dict('saved_processed_percepts.pkl', percept_processed)
    
    if i % 5 == 0:
        save_list_of_dict(f'saved_processed_percepts_backup_v{i}.pkl', percept_processed) # for backup purpose
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)





Generating: 0 -th prompt
Story: Noor is working as a barista at a busy coffee shop.  Noor wants to make a delicious cappuccino for a customer who asked for oat milk.  Noor grabs a milk pitcher and fills it with oat milk.

Main character's belief:  Noor believes that the milk pitcher contains oatmilk.

Event:  A coworker, who didn't hear the customer's request, swaps the oat milk in the pitcher with almond milk while Noor is attending to another task. 

Instructions: Based on the following story and event, generate perceptual cues that the main character might encounter, which MUST challenge their belief. Each cue should be creative, reasonable, and systematically explore the sensory modalities of: visual, auditory, olfactory, gustatory, tactile, thermoception (sense of temperature about objects), proprioception (sense of body position/weight), social perception. Be descriptive and only make vivid descriptions about the perceptual cue, without describing the main character's thoughts. A

In [ ]:
# BigTom new prompt
# random stories

import time 
input_tokens = 0
output_tokens = 0

begin = time.time()
random_percepts = []

for i in range(num_bigtom_stories):
    print("Generating:", i, "-th prompt")
    prompt = f"Story: {new_bigtom.iloc[i]['Story - Context']} {new_bigtom.iloc[i]['Story - Desire']} {new_bigtom.iloc[i]['Story - Perception cues']}\n\n"
    prompt += f"Main character's belief: {new_bigtom.iloc[i]['Story - Belief']}\n\n"
    # prompt = f"Story: {new_bigtom.iloc[i]['Story - Context']} {new_bigtom.iloc[i]['Story - Desire']} {new_bigtom.iloc[i]['Story - Perception cues']} \n\n"
    prompt += f"Causal Event: {new_bigtom.iloc[i]['Story - An external event']} \n\n"
    
    # prompt += f"Instructions: From this Story, generate 10 perceptual cues that {protag_name[i]} may notice about the Event. "
    # prompt += f"Each perceptual cue should continue the Story and Event. Write one sentence for each perceptual cue. "
    # prompt += f"Avoid descriptions of {protag_name[i]}\'s thoughts. Be systematic, reasonable, creative, and innovative, and make the stories diverse (from different contexts). "
    # prompt += f"Avoid making direct comparisons to the item mentioned in the event, but make vivid descriptions about the difference."
    
    # 
    #  visual, auditory, olfactory, gustatory, tactile, 
    prompt += f"Instructions: Based on the causal event, generate perceptual cues that the main character might encounter, which MUST NEGATE {protag_name[i]}'s initial belief and change it to: \"{new_bigtom.iloc[i]['Answers - Belief Aware']}\". "
    prompt += "Each cue should be creative, reasonable, and systematically explore the sensory modalities of: visual, auditory, olfactory, gustatory, tactile, thermoception (sense of temperature about objects), proprioception (sense of body position/weight), social perception. \n"
    prompt += "Be descriptive and only make vivid descriptions about the perceptual cue, without describing the main character's thoughts. Avoid making explicit comparisons between the event and the main character's belief. Generate 1 perceptual cue for each sensory modality. \n"
    prompt += "Be super duper creative! "
    prompt += "Do not use these words: different, difference, instead of, contrasts.\n"
    prompt += "For each cue, write your output in this format: \n"
    prompt += "### Visual Cues:\n1. {cue 1}\n"
    # prompt += "### Visual Cues:\nA bright green label on the milk carton nearby suddenly catches Noor’s eye, glistening under the café's overhead lights. The bold print reads “Almond Milk” in a slightly whimsical font, contrasting sharply with the soft beige of the oat milk packaging Noor is accustomed to seeing.\n\n"


    # prompt += 'Write your output in this template: “### <sense 1> Cues\n1. <perceptual cue (variable length)>\n2. <perceptual cue (variable length)>\n\n### <sense 2> Cues\n1. <perceptual cue (variable length)>\n2. <perceptual cue (variable length)>”'
    # prompt += "Here are some general examples of how perceptual cues should look like, for your reference: "
    # prompt += "1. Maria sees smoke coming from the kitchen, suggesting a fire. Should she change her belief about the kitchen being safe? "
    # prompt += "2. Lila smells gas in the garage, where she believes everything is secure. Should she change her belief about the safety of the garage? "
    
    # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
    input_tokens += count_tokens(prompt)
    

    for _ in range(1): # generate 3 outputs. 
        # print("iteration ")
        output = LM_generation(prompt, temperature = 0.6, max_tokens = 1500, model = "gpt-4o-mini")
        # print(output)
        # put output into the random_percepts
        # print(prompt)
        random_percepts.append(process(output[0], senses, mode = 'random'))
        # print(output[0])
        # testing = output[0].split('\n')
        # print(testing[::3])
        # print(testing[1::3])
        # print(testing[2::3])
        # print(len(output[0].split('\n')))
        # print()
        # print(process(output[0], senses, mode = 'random'))
        output_tokens += count_tokens(output[0])
        if (count_tokens(output[0])>1200):
            print("Warning: long input")
        if (count_tokens(output[0])>1500):
            print("Error: LLM didn't manage to finish. Plz check again")
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
        save_list_of_dict('saved_random_percepts.pkl', random_percepts)
    
    if i % 5 == 0:
        save_list_of_dict(f'saved_random_percepts_backup_v{i}.pkl', random_percepts) # for backup purpose
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)




Generating: 0 -th prompt
Story: Noor is working as a barista at a busy coffee shop.  Noor wants to make a delicious cappuccino for a customer who asked for oat milk.  Noor grabs a milk pitcher and fills it with oat milk.

Main character's belief:  Noor believes that the milk pitcher contains oatmilk.

Causal Event:  A coworker, who didn't hear the customer's request, swaps the oat milk in the pitcher with almond milk while Noor is attending to another task. 

Instructions: Based on the causal event, generate perceptual cues that the main character might encounter, which MUST NEGATE Noor's initial belief and change it to: "Noor believes the milk pitcher contains almond milk.". Each cue should be creative, reasonable, and systematically explore the sensory modalities of: visual, auditory, olfactory, gustatory, tactile, thermoception (sense of temperature about objects), proprioception (sense of body position/weight), social perception. 
Be descriptive and only make vivid descriptions abo

After saving the generated percept cues, we will pass it on for labelling and filtering. Please refer to Part 2 to see how the data will be formatted and prepared for labelling.

**OpenToM**

1. Load and preview the data.

In [1]:
import pandas as pd
# !pip install huggingface_hub

splits = {'Long': 'opentom.json', 'ExtraLong': 'opentom_long.json'}
df = pd.read_json("hf://datasets/SeacowX/OpenToM/" + splits["Long"])

In [ ]:
df.iloc[500]

plot                   Atticus entered the workshop.\nAtticus entered...
plot_info              {'mover': 'Logan', 'eoi': 'jacket', 'original_...
preferences            {'mover': 'Logan hates jacket.', 'observer': '...
personality                               Logan is a considerate person.
sentiment_statement    Although Logan hates jacket, Logan knows that ...
true_sentiment                                         the most positive
intention                  make the jacket easily accessible for Atticus
new_location                                               Atticus' room
observed                                                           False
question               {'question': 'As Atticus, what is your attitud...
narrative              Logan and Atticus were good friends, but they ...
Name: 500, dtype: object

2. Generate percept cues using GPT-4o-mini. You may run the code several times to collect more percept cues. 

In [ ]:

import time 
input_tokens = 0
output_tokens = 0

begin = time.time()
opentom_positive_percepts = []
num_opentom_stories = 121 # number of opentom stories we want to create. 

count = 0
opentom_stories = []
for i, k in enumerate(df['preferences']):
    
    if k['mover_observer'] == '':
        count += 1
        print("Generating:", count, "-th prompt")
        print(f"Story #{i}")
        # 2 options. either observer have false belief or true belief.
        # false belief:
        if k['observer_mover'] != '':
            belief = df.iloc[i]['preferences']['observer_mover']
        else:
            belief = df.iloc[i]['plot_info']['observer'] + ' thinks that '
            belief += df.iloc[i]['preferences']['mover']
            if df.iloc[i]['preferences']['mover'][-1] != '.':
                belief += '.'
        story = df.iloc[i]['preferences']['observer']
        story += ' ' + df.iloc[i]['preferences']['mover']
        if df.iloc[i]['preferences']['mover'][-1] != '.':
            story += '.'
        new_plot = df.iloc[i]['plot'].split('\n')
        new_plot = ' '.join(new_plot[:-1])
        new_preferences = df.iloc[i]['narrative'].split('\n\n')
        if i != 7 and i!= 16 and i!=29 and i!=40:
            new_preferences = ' '.join(new_preferences[2:])
        else:
            new_preferences = ' '.join(new_preferences[1:])
        story += ' ' + new_plot + ' ' + new_preferences
        
        belief_list = belief.split(' ')
        if belief_list[1] == 'mistakenly':
            belief_list.pop(1)
        belief = ' '.join(belief_list)
        
        if belief_list[-2] == 'hates' or belief_list[-2] == 'hate':
            belief_list[-2] = 'dislikes'
            belief = ' '.join(belief_list)
            
            belief_list[-2] = 'likes' # for new belief
        
        elif belief_list[-2] == 'likes' or belief_list[-2] == 'like':
            belief_list[-2] = 'dislikes'
        
        # elif belief_list[-2] == 'hates' or belief_list[-2] == 'hate':
        #     belief_list.pop(-2)
        #     belief_list.insert(-1, 'does not hate')
            
        new_belief = ' '.join(belief_list)
        prompt = f"Story: {story} \n\n"
        prompt += f"Main Character's Belief: {belief} \n\n"
        prompt += f"Instructions: Based on the following story, generate a one-line story consisting an event that MUST NEGATE {df.iloc[i]['plot_info']['observer']}'s initial belief and change it to: \"{new_belief}\" "
        prompt += f"Note that this story MUST also be consistent with {df.iloc[i]['plot_info']['mover']}'s preference, i.e., {df.iloc[i]['preferences']['mover']}. "
        prompt += "Each story should be creative, reasonable, realistic, and systematically explore the sensory modalities of: visual, auditory, olfactory, gustatory, tactile, proprioception (sense of body position/weight). Generate 2 stories for each sensory modality. Be super duper creative! Show, don't tell. Write the stories in this format: \n ### Visual Stories\n1. {story #1}\n2. {story #2}\n "
        opentom_stories.append( {'story': story, 'observer_name':df.iloc[i]['plot_info']['observer'], 'belief':belief, 'mover_name': df.iloc[i]['plot_info']['mover'], 'new_belief':new_belief, 'item':belief_list[-1][:-1]})
        # print(belief_list[-1][:-1])
        # print('Story: ', story)
        # print("Main Character's Belief: ", df.iloc[i]['preferences']['observer_mover'])
        
        print(prompt) #uncomment later

        input_tokens += count_tokens(prompt)
        # true belief
        output = LM_generation(prompt, temperature = 0.9, max_tokens = 1500, model = "gpt-4o-mini")
        # opentom_positive_percepts.append(process(output[0], senses))
        opentom_positive_percepts.append(output[0]) # process later. 
        # print(output[0])
        output_tokens += count_tokens(output[0])
        if (count_tokens(output[0])>1200):
            print("Warning: long input")
        if (count_tokens(output[0])>1500):
            print("Error: LLM didn't manage to finish. Plz check again")
        # save  (add later)
        save_list_of_dict('opentom_positive_percepts.pkl', opentom_positive_percepts)
        
    if count % 5 == 0:
        save_list_of_dict(f'opentom_positive_percepts_backup_v{i}.pkl', opentom_positive_percepts) # for backup purpose
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()
    if count >= num_opentom_stories:
        break

print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)

    
        


Generating: 1 -th prompt
Story #0
Story: Amir likes scarf. Diego hates scarf. Diego entered the patio. Amir entered the patio. Both Diego and Amir noticed that the scarf is in the basket in the patio. In that moment, without exchanging any words, Diego swiftly reached into the basket and snatched the scarf. Amir watched curiously as Diego took a few steps towards a nearby donation bin. With a resolute expression, Diego dropped the scarf into the bin, relieving himself of its presence. And just like that, the scarf that once rested in the patio basket had found a new temporary home in the donation bin, waiting to be discovered by someone who would appreciate its warmth and beauty. Diego turned around to leave the patio, completely unaware that his actions had not gone unnoticed by Amir. 

Main Character's Belief: Amir think that Diego likes scarf. 

Instructions: Based on the following story, generate a one-line story consisting an event that MUST NEGATE Amir's initial belief and change

In [ ]:
# easy neg

import time 
input_tokens = 0
output_tokens = 0

begin = time.time()

opentom_easy_negative_percepts = []


count = 0
opentom_stories = []
for i, k in enumerate(df['preferences']):
    
    if k['mover_observer'] == '':
        count += 1
        print("Generating:", count, "-th prompt")
        print(f"Story #{i}")
        # 2 options. either observer have false belief or true belief.
        # false belief:
        if k['observer_mover'] != '':
            belief = df.iloc[i]['preferences']['observer_mover']
        else:
            belief = df.iloc[i]['plot_info']['observer'] + ' thinks that '
            belief += df.iloc[i]['preferences']['mover']
            if df.iloc[i]['preferences']['mover'][-1] != '.':
                belief += '.'
        story = df.iloc[i]['preferences']['observer']
        story += ' ' + df.iloc[i]['preferences']['mover']
        if df.iloc[i]['preferences']['mover'][-1] != '.':
            story += '.'
        new_plot = df.iloc[i]['plot'].split('\n')
        new_plot = ' '.join(new_plot[:-1])
        new_preferences = df.iloc[i]['narrative'].split('\n\n')
        if i != 7 and i!= 16 and i!=29 and i!=40:
            new_preferences = ' '.join(new_preferences[2:])
        else:
            new_preferences = ' '.join(new_preferences[1:])
        story += ' ' + new_plot + ' ' + new_preferences
        
        belief_list = belief.split(' ')
        if belief_list[1] == 'mistakenly':
            belief_list.pop(1)
        belief = ' '.join(belief_list)
        
        if belief_list[-2] == 'hates' or belief_list[-2] == 'hate':
            belief_list[-2] = 'dislikes'
            belief = ' '.join(belief_list)
            
            belief_list[-2] = 'likes' # for new belief
        
        elif belief_list[-2] == 'likes' or belief_list[-2] == 'like':
            belief_list[-2] = 'dislikes'
        
        # elif belief_list[-2] == 'hates' or belief_list[-2] == 'hate':
        #     belief_list.pop(-2)
        #     belief_list.insert(-1, 'does not hate')
            
        new_belief = ' '.join(belief_list)
        prompt = f"Story: {story} \n\n"
        prompt += f"Main Character's Belief: {belief} \n\n"
        # Based on the following story, generate a one-line story consisting a random event that occurs in the environment, which is unrelated to the object and cannot influence the main character Amir’s belief.
        prompt += f"Instructions: Based on the following story, generate a one-line story consisting a random event that occurs in the environment, which is unrelated to the object and cannot influence the main character {df.iloc[i]['plot_info']['observer']}'s belief."
        prompt += "Each story should be creative, reasonable, realistic, and systematically explore the sensory modalities of: visual, auditory, olfactory, gustatory, tactile, proprioception (sense of body position/weight). Generate 1 story for each sensory modality. Be super duper creative! Show, don't tell. Write the stories in this format: \n ### Visual Stories:\n{story #1}\n\n "
        opentom_stories.append( {'story': story, 'observer_name':df.iloc[i]['plot_info']['observer'], 'belief':belief, 'mover_name': df.iloc[i]['plot_info']['mover'], 'new_belief':new_belief, 'item':belief_list[-1][:-1]})
        # print(belief_list[-1][:-1])
        # print('Story: ', story)
        # print("Main Character's Belief: ", df.iloc[i]['preferences']['observer_mover'])
        print(prompt) #uncomment later
        
#UNCOMMENT LATER
        input_tokens += count_tokens(prompt)
        # true belief
        output = LM_generation(prompt, temperature = 0.9, max_tokens = 1500, model = "gpt-4o-mini")
        # opentom_positive_percepts.append(process(output[0], senses))
        opentom_easy_negative_percepts.append(output[0]) # process later. 
        # print(output[0])
        output_tokens += count_tokens(output[0])
        if (count_tokens(output[0])>1200):
            print("Warning: long input")
        if (count_tokens(output[0])>1500):
            print("Error: LLM didn't manage to finish. Plz check again")
        # save  (add later)
        save_list_of_dict('opentom_easy_negative_percepts.pkl', opentom_easy_negative_percepts)
        
    if count % 5 == 0:
        save_list_of_dict(f'opentom_easy_negative_percepts_backup_v{i}.pkl', opentom_easy_negative_percepts) # for backup purpose
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()
    if count >= num_opentom_stories: 
        break

print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)

        


Generating: 1 -th prompt
Story #0
Story: Amir likes scarf. Diego hates scarf. Diego entered the patio. Amir entered the patio. Both Diego and Amir noticed that the scarf is in the basket in the patio. In that moment, without exchanging any words, Diego swiftly reached into the basket and snatched the scarf. Amir watched curiously as Diego took a few steps towards a nearby donation bin. With a resolute expression, Diego dropped the scarf into the bin, relieving himself of its presence. And just like that, the scarf that once rested in the patio basket had found a new temporary home in the donation bin, waiting to be discovered by someone who would appreciate its warmth and beauty. Diego turned around to leave the patio, completely unaware that his actions had not gone unnoticed by Amir. 

Main Character's Belief: Amir think that Diego likes scarf. 

Instructions: Based on the following story, generate a one-line story consisting a random event that occurs in the environment, which is un

3. Error Handling for problematic LLM outputs.

In [192]:


# preview
# Load labelled data
import math 
positive_percepts_dict = load_list_of_dict('opentom_positive_percepts.pkl')
positive_percepts = []
for line in positive_percepts_dict:
    split_line = line.split('\n')
    if len(split_line)>31: # 32 34 39
        line = line.replace('\n2.', '2.')
        while(True):
            new_line = line.replace(' 2.', '2.')
            if (new_line == line):
                line = new_line
                break
            line = new_line
        split_line = line.split('\n')
    if len(split_line) == 27: # 27, 26 (problem. basically 23. missed proprioception), 22 (missed thermoception), 23
        line += '\n\n### Social Perception Stories\n1. NaN\n2. NaN'
        split_line = line.split('\n')
        
    if len(split_line) == 26:
        line = line.split('\n\n### Proprioception Stories')[0]
        split_line = line.split('\n')
    
    if len(split_line) == 23:
        line += '\n\n### Proprioception Stories\n1. NaN\n2. NaN\n\n### Social Perception Stories\n1. NaN\n2. NaN'
        split_line = line.split('\n')
    
    if len(split_line)<23:
        line = line.split('\n\n### Thermoception Stories')[0]
        line += '\n\n### Thermoception Stories\n1. NaN\n2. NaN\n\n### Proprioception Stories\n1. NaN\n2. NaN\n\n### Social Perception Stories\n1. NaN\n2. NaN'
        # split_line = line.split('\n')
        # print(len(split_line))
        # for l in split_line:
        #     print(l)
        # print(split_line)
        # print(len(split_line))
        split_line = line.split('\n')
    # split_line = line.split('\n')
    
    if len(split_line) != 31: # checking
        print(len(split_line))
        print('Error')
    processed_line = process(line, senses, mode = 'default')
    positive_percepts.append(processed_line)


In [207]:
easy_negative_percepts_dict = load_list_of_dict('opentom_easy_negative_percepts.pkl')
easy_negative_percepts = []
for i, line in enumerate(easy_negative_percepts_dict):
    split_line = line.split('\n')
    
    if len(split_line)<23:
        line += '\n\n### Social Perception Stories:\nNaN'
        split_line = line.split('\n')
        
    if len(split_line) != 23: # checking
        print(len(split_line))
        for l in split_line:
            print(l)
        print('Error')
        print(i)
    processed_line = process(line, senses, mode = 'random')
    easy_negative_percepts.append(processed_line)

len(easy_negative_percepts)
print(easy_negative_percepts[0])

{'visual': "A bright red cardinal flitted from branch to branch in the nearby oak tree, its feathers flashing in the sunlight as it chirped melodiously, catching Amir's eye as he unconsciously followed the flickering movement. ", 'auditory': "The faint sound of laughter floated in from a distant gathering, intertwining with the rustling leaves, creating a symphony of joy that filled Amir's ears as he stood silently, captivated by the moment.", 'olfactory': "A warm waft of freshly baked bread from a nearby café enveloped Amir, mingling with the scent of blooming jasmine, making his stomach rumble with a longing for a savory bite, all while he remained oblivious to Diego's actions.", 'gustatory': "As Amir absentmindedly licked his lips, he reminisced about the tangy sweetness of the lemonade he'd enjoyed earlier, feeling the refreshing taste dance on his tongue, unaware of the scarf's swift exit.", 'tactile': "A gentle breeze brushed against Amir's skin, carrying with it the crispness of

After saving the generated percept cues, we will pass it on for labelling and filtering. Please refer to Part 2 to see how the data will be formatted and prepared for labelling.

**Part 2. Prepare the data for labelling in MechanicalTurk**

1. Adjust the formatting of both datasets and turn the raw data into CSV format. 

In [ ]:
story_list = []
for i in range(num_bigtom_stories): 
    s = f"Story: {new_bigtom.iloc[i]['Story - Context']} {new_bigtom.iloc[i]['Story - Desire']} {new_bigtom.iloc[i]['Story - Perception cues']}\n\n"
    s += f"Main character's belief: {new_bigtom.iloc[i]['Story - Belief']}\n\n"
    s += f"Event: {new_bigtom.iloc[i]['Story - An external event']} \n\n"
    story_list.append(s)

In [ ]:
# BigToM
bigtom_easy_neg = load_list_of_dict('saved_random_percepts.pkl')
bigtom_easy_neg = pd.DataFrame(bigtom_easy_neg)
bigtom_easy_neg['story'] = story_list[0:100]
bigtom_easy_neg = bigtom_easy_neg.transpose()
bigtom_easy_neg.to_csv(f'bigtom_easy_negative_percepts.csv') 

In [245]:
bigtom_easy_neg

,visual,auditory,olfactory,gustatory,tactile,thermo,proprio,social,story
0,A vibrant splash of color erupts as the decora...,"A sharp, echoing clatter reverberates through ...","As the mug shatters, a waft of freshly ground ...",The taste of the air shifts as a hint of spill...,"The cool, smooth handle of the milk pitcher fe...",The oat milk in the pitcher retains a chill fr...,Noor senses the subtle shift in her stance as ...,The expressions of the customers shift from cu...,Story: Noor is working as a barista at a busy ...
1,A cascade of shimmering droplets escapes the g...,"A sharp gasp escapes from a nearby visitor, fo...",The crisp scent of the freshly painted artwork...,A slight taste of metallic tang lingers on Zar...,The coolness of the polished floor beneath Zar...,A sudden gust of air from the gallery’s ventil...,Zara feels the slight shift in her stance as s...,The collective murmur of the crowd around her ...,Story: Zara is attending an art exhibition at ...
2,A vibrant plume of feathers bursts into view a...,The distinct clucking and gobbling of the wild...,"A rich, earthy aroma wafts through the air, mi...","As Carlos wipes his brow, a sudden gust of win...",The rough texture of the corn stalks brushes a...,The sun beats down on the back of Carlos's nec...,As Carlos shifts his weight from one foot to a...,"The wild turkeys move in a curious flock, thei...",Story: Carlos is a farmer in a small village i...
3,A vibrant splash of orange and blue fills Kofi...,The cheerful shouts and laughter of the childr...,The salty tang of the ocean air is layered wit...,"As Kofi’s mouth waters, he can almost taste th...",The coarse texture of the sand beneath Kofi's ...,"The sun casts a warm embrace around Kofi, its ...",Kofi feels the weight of his fishing net resti...,As the children cheer and high-five each other...,Story: Kofi is a fisherman from a small villag...
4,The vibrant colors of the dragon dance costume...,The rhythmic sound of drums resonates through ...,The rich scent of incense wafts through the ai...,The tangy flavor of pickled vegetables bursts ...,"The cool breeze brushes against the skin, a ge...",The sun beats down with a gentle warmth that e...,"As Azola stands at the edge of the gathering, ...",The laughter and chatter of villagers create a...,Story: Azola is a teacher at a small school in...
...,...,...,...,...,...,...,...,...,...
95,"The local artist steps into the workshop, a wh...",The soft sound of the artist’s paintbrushes cl...,"A warm, earthy scent wafts through the room as...","As the artist takes a sip from a travel mug, t...",The artist's fingers brush against the rough s...,"A gentle draft flows through the workshop, con...",As the artist shifts their weight from one foo...,The artist’s enthusiastic gestures punctuate t...,Story: Amara is a sculptor in a small town in ...
96,The reporter’s notepad is a kaleidoscope of co...,The soft chime of a small bell rings out as th...,"The warm, buttery aroma of freshly baked pastr...",A delicate sample of lemon curd tart rests on ...,"The cool, smooth surface of the cake mold feel...",The warmth from the oven radiates through the ...,Mei shifts her weight from one foot to the oth...,The reporter's eyes light up with genuine admi...,"Story: Mei, a talented pastry chef from a smal..."
97,"The rare ingredient, a vibrant crimson chili p...",As the fellow chef hands over the chili pepper...,"A sharp, spicy aroma wafts from the chili pepp...","The moment Mei slices into the chili, a burst ...","As Mei holds the chili pepper, its smooth, sli...","The wok, hanging just above the stove, radiate...","As Mei reaches for the chili pepper, she sense...","The fellow chef stands nearby, a broad smile i...","Story: Mei, a talented chef in a small Chinese..."
98,A vibrant array of fruits spills out from the ...,As the basket lands softly on the kitchen coun...,"A burst of sweet, tangy fragrance wafts throug...",Jia bites into a juicy slice of the freshly gi...,"The smooth

In [ ]:
all_percepts = [positive_percepts, easy_negative_percepts]
percept_names = ['positive', 'easy_negative']
for idx, percept in enumerate(all_percepts):
    for i in range(121):
    # for i in [0]: # testing
        df = pd.DataFrame(percept)
        # post_processing
        for sense in senses:
            df[f'{sense}_{1}'] = [df[sense][j][0] for j in range(121)]
            df[f'{sense}_{2}'] = [df[sense][j][1] for j in range(121)]
        df = df.drop(columns=senses)
        df['story'] = [k['story'] for k in opentom_stories]
        df['initial belief'] = [k['belief'] for k in opentom_stories]
        df['new belief'] = [k['new_belief'] for k in opentom_stories]
        df = df.transpose()

        df.to_csv(f'opentom_{percept_names[idx]}_percepts.csv') 

2. Generate distractor answer options for our MCQA benchmark. We want to have 4 choices for each question, but we only have 2 right now. 

In [ ]:
# Generate Story Template
# i = 1
import time 
input_tokens = 0
output_tokens = 0

begin = time.time()
bigtom_distractor = []

for i in range(num_bigtom_stories):
    print("Generating:", i, "-th prompt")
    prompt = f"Instruction: Based on these two given sentences, generate a third sentence in a similar format, with similar content. "
    prompt += "Avoid paraphrasing the given sentences and generate sentences that differ in meaning as the first two: \n"
    prompt += f"1. {new_bigtom.iloc[i]['Answers - Belief Aware']}\n"
    prompt += f"2. {new_bigtom.iloc[i]['Answers - Belief not Aware']}\n"
    prompt += "Return your answer in the format: 3. \{sentence\}"
    # print(prompt)
    
    input_tokens += count_tokens(prompt)
    
    # print(prompt)
    for _ in range(1):
        # print("iteration ")
        max_tokens = 100
        output = LM_generation(prompt, temperature = 0.6, max_tokens = max_tokens, model = "gpt-4o-mini")
        # print(output)
        # put output into the percept_processed
        bigtom_distractor.append(output[0])
        output_tokens += count_tokens(output[0])
        if (count_tokens(output[0])>70):
            print("Warning: long input")
        if (count_tokens(output[0])>max_tokens):
            print("Error: LLM didn't manage to finish. Plz check again")
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
        save_list_of_dict('new_bigtom_distractor.pkl', bigtom_distractor)
    
    if i % 5 == 0:
        save_list_of_dict(f'new_bigtom_distractor_v{i}.pkl', bigtom_distractor) # for backup purpose
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)





Generating: 0 -th prompt
Time taken for these 5 stories: 1.214639663696289 seconds.
Generating: 1 -th prompt
Generating: 2 -th prompt
Generating: 3 -th prompt
Generating: 4 -th prompt
Generating: 5 -th prompt
Time taken for these 5 stories: 3.485103130340576 seconds.
Generating: 6 -th prompt
Generating: 7 -th prompt
Generating: 8 -th prompt
Generating: 9 -th prompt
Generating: 10 -th prompt
Time taken for these 5 stories: 4.24028205871582 seconds.
Generating: 11 -th prompt
Generating: 12 -th prompt
Generating: 13 -th prompt
Generating: 14 -th prompt
Generating: 15 -th prompt
Time taken for these 5 stories: 3.693131923675537 seconds.
Generating: 16 -th prompt
Generating: 17 -th prompt
Generating: 18 -th prompt
Generating: 19 -th prompt
Generating: 20 -th prompt
Time taken for these 5 stories: 3.4953410625457764 seconds.
Generating: 21 -th prompt
Generating: 22 -th prompt
Generating: 23 -th prompt
Generating: 24 -th prompt
Generating: 25 -th prompt
Time taken for these 5 stories: 3.13023

In [ ]:
# Generate Story Template
# i = 1
import time 
input_tokens = 0
output_tokens = 0

begin = time.time()

option_four = []

for i in range(num_bigtom_stories): # skip 0 cuz we already did it. 
    print("Generating:", i, "-th prompt")
    prompt = f"Question: Select the answer option that best describes {protag_name[i]}\'s belief.\n"
    prompt += f"Answer Options: \n"
    prompt += f"1. {new_bigtom.iloc[i]['Answers - Belief Aware']}\n"
    prompt += f"2. {new_bigtom.iloc[i]['Answers - Belief not Aware']}\n"
    prompt += f"{bigtom_distractor[i]}\n"
    # prompt += f"Instruction: Based on this question-answer set, generate a general answer option that says: {protag_name[i]} does not know ...\n"
    prompt += f"Instruction: Based on this question-answer set, generate a general answer option that says: {protag_name[i]} believes [pronoun] does not know ...\n"

    # print(prompt)
    
    input_tokens += count_tokens(prompt)
    
    # print(prompt)
    for _ in range(1): # generate 3 outputs. 
        # print("iteration ")
        max_tokens = 100
        output = LM_generation(prompt, temperature = 0.6, max_tokens = max_tokens, model = "gpt-4o-mini")
        # print(output)
        # put output into the percept_processed
        option_four.append(output[0])
        output_tokens += count_tokens(output[0])
        if (count_tokens(output[0])>70):
            print("Warning: long input")
        if (count_tokens(output[0])>max_tokens):
            print("Error: LLM didn't manage to finish. Plz check again")
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
        save_list_of_dict('new_option_four.pkl', option_four)
    
    if i % 5 == 0:
        save_list_of_dict(f'new_option_four_v{i}.pkl', option_four) # for backup purpose
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


# print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
# print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)

Generating: 0 -th prompt
Time taken for these 5 stories: 0.7451598644256592 seconds.
Generating: 1 -th prompt
Generating: 2 -th prompt
Generating: 3 -th prompt
Generating: 4 -th prompt
Generating: 5 -th prompt
Time taken for these 5 stories: 4.001812934875488 seconds.
Generating: 6 -th prompt
Generating: 7 -th prompt
Generating: 8 -th prompt
Generating: 9 -th prompt
Generating: 10 -th prompt
Time taken for these 5 stories: 3.2075858116149902 seconds.
Generating: 11 -th prompt
Generating: 12 -th prompt
Generating: 13 -th prompt
Generating: 14 -th prompt
Generating: 15 -th prompt
Time taken for these 5 stories: 3.7460498809814453 seconds.
Generating: 16 -th prompt
Generating: 17 -th prompt
Generating: 18 -th prompt
Generating: 19 -th prompt
Generating: 20 -th prompt
Time taken for these 5 stories: 3.2795231342315674 seconds.
Generating: 21 -th prompt
Generating: 22 -th prompt
Generating: 23 -th prompt
Generating: 24 -th prompt
Generating: 25 -th prompt
Time taken for these 5 stories: 3.5

In [ ]:
# For formatting purpose.
for i in range(100):
    bigtom_distractor[i] = bigtom_distractor[i].lstrip('3. ') 
    option_four[i] = option_four[i].lstrip('4. ')

In [374]:
for i in range(100):
    print(i)
    print(f"1. {new_bigtom.iloc[i]['Answers - Belief Aware']}")
    print(f"2. {new_bigtom.iloc[i]['Answers - Belief not Aware']}")
    print(f"3. {bigtom_distractor[i]}")
    print(f"4. {option_four[i]}")

0
1. Noor believes the milk pitcher contains almond milk.
2. Noor believes the milk pitcher contains oat milk.
3. Noor believes the milk pitcher contains soy milk.
4. Noor believes she does not know what type of milk is in the pitcher.
1
1. Zara believes the painting is the work of an emerging artist.
2. Zara believes the painting is the work of a famous artist.
3. Zara believes the painting is the work of a local artisan.
4. Zara believes she does not know the identity of the artist.
2
1. Carlos believes the water valve is open.
2. Carlos believes the water valve is closed.
3. Carlos believes the water valve is malfunctioning.
4. Carlos believes he does not know whether the water valve is open or closed.
3
1. Kofi believes his fishing net is torn apart.
2. Kofi believes his fishing net is in good condition.
3. Kofi believes his fishing net is missing entirely.
4. Kofi believes he does not know the condition of his fishing net.
4
1. Azola believes the package contains a hand-knit scarf

3. Convert the MCQA dataset into structured format.

In [ ]:
# Load labelled data
import math 
bigtom_pos_and_hard_neg = pd.read_csv("bigtom_new_pos&hard_neg.csv")
bigtom_pos_and_hard_neg = bigtom_pos_and_hard_neg.transpose()
bigtom_pos_and_hard_neg.columns = bigtom_pos_and_hard_neg.iloc[0]
bigtom_pos_and_hard_neg = bigtom_pos_and_hard_neg.drop(bigtom_pos_and_hard_neg.index[0])
# for _ in range(3):
#     bigtom_pos_and_hard_neg = bigtom_pos_and_hard_neg.drop(bigtom_pos_and_hard_neg.index[100])
bigtom_pos_and_hard_neg = bigtom_pos_and_hard_neg.drop(columns = ['story'])

bigtom_pos_and_hard_neg

In [ ]:
#JSON (store as list of dicts. jsonl)
# ID, tag of label (pos/hard_neg, easy_neg), story, instance, sense, initial belief, answer_options.
# ID format: <cue> - <tag of label> - id
my_bigtom=[]
for i in range(100):
    idx = 0
    for col in bigtom_pos_and_hard_neg.iloc[i].keys():
        if str(bigtom_pos_and_hard_neg.iloc[i][col]) == 'nan':
            continue
        elif str(bigtom_pos_and_hard_neg.iloc[i][col]) == 'NaN':
            continue
        instance = {}
        sense = col.split('_')[0]
        instance_id = f"{i}-0-{idx}"
        idx += 1
        instance['ID'] = instance_id
        instance['category'] = 'pos/hard_neg'
        instance['sense'] = sense
        instance['story'] = f"{new_bigtom.iloc[i]['Story - Context']} {new_bigtom.iloc[i]['Story - Desire']} {new_bigtom.iloc[i]['Story - Perception cues']}"
        instance['event'] = f"{new_bigtom.iloc[i]['Story - An external event']}"
        instance['protag_name'] = protag_name[i]
        instance['percept_cue'] = bigtom_pos_and_hard_neg.iloc[i][col]
        instance['initial_belief'] = new_bigtom.iloc[i]['Story - Belief']
        instance['default_question'] = f"After the event happened, based on the main character {protag_name[i]}'s perception of the event, which of the following statements best describes {protag_name[i]}'s new belief?"
        instance['answer_options'] = [new_bigtom.iloc[i]['Answers - Belief not Aware'],new_bigtom.iloc[i]['Answers - Belief Aware'], bigtom_distractor[i], option_four[i]]# list. 
        my_bigtom.append(instance)
        
my_bigtom

In [ ]:
# Load labelled data
import math 
bigtom_easy_neg = pd.read_csv("bigtom_new_easy_neg.csv")
bigtom_easy_neg = bigtom_easy_neg.transpose()
bigtom_easy_neg.columns = bigtom_easy_neg.iloc[0]
bigtom_easy_neg = bigtom_easy_neg.drop(bigtom_easy_neg.index[0])
# for _ in range(3):
#     bigtom_pos_and_hard_neg = bigtom_pos_and_hard_neg.drop(bigtom_pos_and_hard_neg.index[100])
bigtom_easy_neg = bigtom_easy_neg.drop(columns = ['story'])



In [ ]:
my_bigtom_neg=[]
for i in range(100):
    idx = 0
    for col in bigtom_easy_neg.iloc[i].keys():
        if str(bigtom_easy_neg.iloc[i][col]) == 'nan':
            continue
        elif str(bigtom_easy_neg.iloc[i][col]) == 'NaN':
            continue
        instance = {}
        sense = col #.split('_')[0]
        instance_id = f"{i}-1-{idx}"
        idx += 1
        instance['ID'] = instance_id
        instance['category'] = 'easy_neg'
        instance['sense'] = sense
        instance['story'] = f"{new_bigtom.iloc[i]['Story - Context']} {new_bigtom.iloc[i]['Story - Desire']} {new_bigtom.iloc[i]['Story - Perception cues']}"
        instance['event'] = f"{new_bigtom.iloc[i]['Story - An external event']}"
        instance['protag_name'] = protag_name[i]
        instance['percept_cue'] = bigtom_easy_neg.iloc[i][col]
        instance['initial_belief'] = new_bigtom.iloc[i]['Story - Belief']
        instance['default_question'] = f"After the event happened, based on the main character {protag_name[i]}'s perception of the event, which of the following statements best describes {protag_name[i]}'s new belief?"
        instance['answer_options'] = [new_bigtom.iloc[i]['Answers - Belief not Aware'],new_bigtom.iloc[i]['Answers - Belief Aware'], bigtom_distractor[i], option_four[i]]# list. 
        my_bigtom_neg.append(instance)
        
my_bigtom_neg

In [410]:
my_bigtom += my_bigtom_neg

Save a JSON version.

In [413]:
import json
with open("mybigtom.jsonl", "w") as file:
    for item in my_bigtom:
        json_string = json.dumps(item)
        file.write(json_string + '\n')

4. Create CSV file for MechanicalTurk

In [454]:
# create bigtom csv from json: keys: 
import csv
csv_file = open('my_bigtom_csv.csv', 'w', encoding='utf-8')
writer = csv.writer(csv_file, delimiter = ',')
col_names = list(my_bigtom[0].keys())[:-1]
for i in range(4):
    col_names.append(f'answer_option_{i}')
# print(len(col_names))
writer.writerow(col_names)
for entry in my_bigtom:
    line_contents = list(entry.values())[:-1]
    line_contents = [f"{l}" for l in line_contents]
    for i in range(4):
        line_contents.append(f"\"{entry['answer_options'][i]}\"")
    # print(line_contents)
    # print(len(line_contents))
    writer.writerow(line_contents)
csv_file.close()

5. Repeat the same steps for OpenToM

In [ ]:
# Load labelled data
import math 
opentom_pos_and_hard_neg = pd.read_csv("opentom_new_pos&hard_neg.csv")
opentom_pos_and_hard_neg = opentom_pos_and_hard_neg.transpose()
opentom_pos_and_hard_neg.columns = opentom_pos_and_hard_neg.iloc[0]
opentom_pos_and_hard_neg = opentom_pos_and_hard_neg.drop(opentom_pos_and_hard_neg.index[0])
# for _ in range(3):
#     bigtom_pos_and_hard_neg = bigtom_pos_and_hard_neg.drop(bigtom_pos_and_hard_neg.index[100])
opentom_pos_and_hard_neg = opentom_pos_and_hard_neg.drop(columns = ['story', 'initial belief', 'new belief'])



# positives_cols = labelled_percepts.keys()[::2]
# hard_negatives_cols = labelled_percepts.keys()[1::2]

# positives = [ {col.split('_')[0]: labelled_percepts.iloc[story_id][col]  for col in positives_cols} for story_id in range(100)]
# hard_negatives = [ {col.split('_')[0]: labelled_percepts.iloc[story_id][col]  for col in hard_negatives_cols} for story_id in range(100)]
opentom_pos_and_hard_neg

Generate third and fourth answer options for the MCQA datset

In [495]:
opentom_random_distractor = []
for i in range(121):
    split_form = opentom_stories[i]['story'].split('entered the')
    if len(split_form)==1:
        split_form = opentom_stories[i]['story'].split('is in the')
    if len(split_form) == 1:
        print(i, split_form)
    distractor = split_form[1]
    distractor = distractor.split('.')[0]
    distractor = distractor.split(' in the')[0]
    opentom_random_distractor.append(distractor)

In [497]:
# third option
opentom_third_answer = []
opentom_fourth_answer = []
for i in range(121):
    opentom_third_answer.append(f"{opentom_stories[i]['observer_name']} does not know if {opentom_stories[i]['mover_name']} likes {opentom_stories[i]['item']} or not.")
    fourth_answer = opentom_stories[i]['belief'].split(' ')[:-1]
    fourth_answer.append(opentom_random_distractor[i].lstrip(' '))
    fourth_answer = ' '.join(fourth_answer)
    opentom_fourth_answer.append(fourth_answer)

In [498]:
for i in range(121):
    print(i)
    print(f"1. {opentom_stories[i]['belief']}")
    print(f"2. {opentom_stories[i]['new_belief']}")
    print(f"3. {opentom_third_answer[i]}")
    print(f"4. {opentom_fourth_answer[i]}.")

0
1. Amir think that Diego likes scarf.
2. Amir think that Diego dislikes scarf.
3. Amir does not know if Diego likes scarf or not.
4. Amir think that Diego likes patio.
1
1. Paxton thinks that Hadley dislikes hat.
2. Paxton thinks that Hadley likes hat.
3. Paxton does not know if Hadley likes hat or not.
4. Paxton thinks that Hadley dislikes pantry.
2
1. Marcos think that Abraham likes melon.
2. Marcos think that Abraham dislikes melon.
3. Marcos does not know if Abraham likes melon or not.
4. Marcos think that Abraham likes office.
3
1. Jayden think that Jeremy likes peas.
2. Jayden think that Jeremy dislikes peas.
3. Jayden does not know if Jeremy likes peas or not.
4. Jayden think that Jeremy likes garden.
4
1. Richard thinks that Andrew likes hat.
2. Richard thinks that Andrew dislikes hat.
3. Richard does not know if Andrew likes hat or not.
4. Richard thinks that Andrew likes back_yard.
5
1. Manuel think that Ariana likes shirt.
2. Manuel think that Ariana dislikes shirt.
3. Man

In [489]:
f = open("opentom_stories_text.txt", "w")
f.write("\n".join([opentom_stories[i]['story'] for i in range(121)]))
f.close()


In [490]:

#open and read the file after the appending:
f = open("opentom_stories_text.txt", "r")
processed_opentom_stories = f.read()

In [492]:
processed_opentom_stories = processed_opentom_stories.split('\n')

In [ ]:
#JSON (store as list of dicts. jsonl)
# ID, tag of label (pos/hard_neg, easy_neg), story, instance, sense, initial belief, answer_options.
# ID format: <cue> - <tag of label> - id
my_opentom=[]
for i in range(num_opentom_stories):
    idx = 0
    for col in opentom_pos_and_hard_neg.iloc[i].keys():
        if str(opentom_pos_and_hard_neg.iloc[i][col]) == 'nan':
            continue
        elif str(opentom_pos_and_hard_neg.iloc[i][col]) == 'NaN':
            continue
        instance = {}
        sense = col.split('_')[0]
        instance_id = f"{i}-0-{idx}"
        idx += 1
        instance['ID'] = instance_id
        instance['category'] = 'pos/hard_neg'
        instance['sense'] = sense
        instance['story'] = processed_opentom_stories[i]
        instance['protag_name'] = opentom_stories[i]['observer_name']
        instance['percept_cue'] = opentom_pos_and_hard_neg.iloc[i][col]
        instance['initial_belief'] = opentom_stories[i]['belief']
        instance['default_question'] = f"After the story happened, based on the percept cue experienced by the main character {opentom_stories[i]['observer_name']}, which of the following statements best describes {opentom_stories[i]['observer_name']}'s new belief about {opentom_stories[i]['mover_name']}'s preferences?"
        instance['answer_options'] = [opentom_stories[i]['belief'], opentom_stories[i]['new_belief'], opentom_third_answer[i], opentom_fourth_answer[i]]
        my_opentom.append(instance)
        
        

In [536]:
# Load labelled data
import math 
opentom_easy_neg = pd.read_csv("opentom_new_easy_neg.csv")
opentom_easy_neg = opentom_easy_neg.transpose()
opentom_easy_neg.columns = opentom_easy_neg.iloc[0]
opentom_easy_neg = opentom_easy_neg.drop(opentom_easy_neg.index[0])
# for _ in range(3):
#     bigtom_pos_and_hard_neg = bigtom_pos_and_hard_neg.drop(bigtom_pos_and_hard_neg.index[100])
opentom_easy_neg = opentom_easy_neg.drop(columns = ['story', 'initial belief', 'new belief'])



In [ ]:
my_opentom_neg=[]
for i in range(num_opentom_stories):
    idx = 0
    for col in opentom_easy_neg.iloc[i].keys():
        if str(opentom_easy_neg.iloc[i][col]) == 'nan':
            continue
        elif str(opentom_easy_neg.iloc[i][col]) == 'NaN':
            continue
        instance = {}
        sense = col
        instance_id = f"{i}-1-{idx}"
        idx += 1
        instance['ID'] = instance_id
        instance['category'] = 'easy_neg'
        instance['sense'] = sense
        instance['story'] = processed_opentom_stories[i]
        instance['protag_name'] = opentom_stories[i]['observer_name']
        instance['percept_cue'] = opentom_easy_neg.iloc[i][col]
        instance['initial_belief'] = opentom_stories[i]['belief']
        instance['default_question'] = f"After the story happened, based on the percept cue experienced by the main character {opentom_stories[i]['observer_name']}, which of the following statements best describes {opentom_stories[i]['observer_name']}'s new belief about {opentom_stories[i]['mover_name']}'s preferences?"
        instance['answer_options'] = [opentom_stories[i]['belief'], opentom_stories[i]['new_belief'], opentom_third_answer[i], opentom_fourth_answer[i]]
        my_opentom_neg.append(instance)
        

        
        

In [ ]:
import random
random.seed(0)
sampled_opentom_neg = random.sample(my_opentom_neg, 520) # we do sampling - there is no need to keep all the negative samples. 
sampled_opentom_neg

In [541]:
import json
with open("sampled_opentom_neg.jsonl", "w") as file:
    for item in sampled_opentom_neg:
        json_string = json.dumps(item)
        file.write(json_string + '\n')

In [542]:
my_opentom += sampled_opentom_neg

In [544]:
import json
with open("my_opentom.jsonl", "w") as file:
    for item in my_opentom:
        json_string = json.dumps(item)
        file.write(json_string + '\n')

In [545]:
# create bigtom csv from json: keys: 
import csv
csv_file = open('my_opentom_csv.csv', 'w', encoding='utf-8')
writer = csv.writer(csv_file, delimiter = ',')
col_names = list(my_opentom[0].keys())[:-1]
for i in range(4):
    col_names.append(f'answer_option_{i}')
# print(len(col_names))
writer.writerow(col_names)
for entry in my_opentom:
    line_contents = list(entry.values())[:-1]
    line_contents = [f"{l}" for l in line_contents]
    for i in range(4):
        line_contents.append(f"\"{entry['answer_options'][i]}\"")
    # print(line_contents)
    # print(len(line_contents))
    writer.writerow(line_contents)
csv_file.close()

The CSV files will then be passed on for labelling in MechanicalTurk. Refer to the "labelling" folder for more information.

**Part 3. Labelling and Filtering**

Labelling is done in a separate tool by several experts: MechanicalTurk.
Filtering is primarily performed manually with several experts, on the basis of removing instances that are flagged as "problematic" and discussing on ambiguous samples (i.e., with different labels).

1. Open Labelled Samples

In [1]:
import json
with open('llm_ready_bigtom_vfin.jsonl') as f: # ambiguous story
    final_bigtom_new = [json.loads(line) for line in f]

with open('llm_ready_opentom_vfin.jsonl') as f: # ambiguous story
    final_opentom_new = [json.loads(line) for line in f]

In [2]:
import numpy as np
with open('bigtom_filter.npy', 'rb') as f:
    bigtom_right_labels = np.load(f)
with open('opentom_filter.npy', 'rb') as f:
    opentom_right_labels = np.load(f)

**Part 4. Calculate Inter-Annotator Agreement.**
Here, we will use Fleiss Kappa.

For Task 1

In [3]:
from statsmodels.stats.inter_rater import fleiss_kappa
import krippendorff
import numpy as np 

# need to calculate interagreement for Task 1 (is there belief change).
# [have change, no change] - [# of 1s, # of 0s]
bigtom_change_new = [[round(3*instance['belief_change_label_raw']), 3 - round(3*instance['belief_change_label_raw'])] for instance in np.array(final_bigtom_new)[bigtom_right_labels]]
opentom_change_new = [[round(3*instance['belief_change_label_raw']), 3 - round(3*instance['belief_change_label_raw'])] for instance in np.array(final_opentom_new)[opentom_right_labels]]

bigtom_change_new = np.array(bigtom_change_new)
opentom_change_new = np.array(opentom_change_new)

# Now, calculate label inter-agreement (Fleiss kappa)
num_annotators = 3
bigtom_p_j = bigtom_change_new.mean(axis = 0) / num_annotators # find probabilities of each class
bigtom_p_j = bigtom_p_j * (1/bigtom_p_j.sum()) # scale a bit
opentom_p_j = opentom_change_new.mean(axis = 0) / num_annotators
opentom_p_j = opentom_p_j * (1/opentom_p_j.sum()) # scale a bit
# print(opentom_p_j)

bigtom_P_i = ((bigtom_change_new ** 2).sum(axis = 1) - num_annotators)/(num_annotators * (num_annotators - 1))
opentom_P_i = ((opentom_change_new ** 2).sum(axis = 1) - num_annotators)/(num_annotators * (num_annotators - 1))

bigtom_P_bar = bigtom_P_i.mean()
bigtom_P_e_bar = (bigtom_p_j ** 2).sum()
opentom_P_bar = opentom_P_i.mean()
opentom_P_e_bar = (opentom_p_j ** 2).sum()

bigtom_kappa = (bigtom_P_bar - bigtom_P_e_bar) / (1 - bigtom_P_e_bar)
opentom_kappa = (opentom_P_bar - opentom_P_e_bar) / (1 - opentom_P_e_bar)

print('Kappa for Task 1: ', bigtom_kappa)
print('Kappa for Task 1: ', opentom_kappa)

bigtom_kappa = fleiss_kappa(bigtom_change_new, method='fleiss')
opentom_kappa = fleiss_kappa(opentom_change_new, method='fleiss')
print('Fleiss Kappa for change to what task: ', bigtom_kappa)
print('Fleiss Kappa for change to what task: ', opentom_kappa)


Kappa for Task 1:  0.7757764144141691
Kappa for Task 1:  0.7417675536179354
Fleiss Kappa for change to what task:  0.7757764144141691
Fleiss Kappa for change to what task:  0.7417675536179354


For Task 2

In [4]:
# Now, we can calculate label inter-agreement - Task 2 
from statsmodels.stats.inter_rater import fleiss_kappa
import krippendorff
import numpy as np 
options = ['A', 'B', 'C', 'D']

bigtom_annotate_new = [[instance['new_belief_label_raw'].count(o) for o in options] for instance in np.array(final_bigtom_new)[bigtom_right_labels]]
opentom_annotate_new = [[instance['new_belief_label_raw'].count(o) for o in options] for instance in np.array(final_opentom_new)[opentom_right_labels]]

bigtom_annotate_new = np.array(bigtom_annotate_new)
opentom_annotate_new = np.array(opentom_annotate_new)
# Now, calculate label inter-agreement (Fleiss kappa)
num_annotators = 3
bigtom_p_j = bigtom_annotate_new.mean(axis = 0) / num_annotators # find probabilities of each class
bigtom_p_j = bigtom_p_j * (1/bigtom_p_j.sum()) # scale a bit
opentom_p_j = opentom_annotate_new.mean(axis = 0) / num_annotators
opentom_p_j = opentom_p_j * (1/opentom_p_j.sum()) # scale a bit

# print(bigtom_p_j)

bigtom_P_i = ((bigtom_annotate_new ** 2).sum(axis = 1) - num_annotators)/(num_annotators * (num_annotators - 1))
opentom_P_i = ((opentom_annotate_new ** 2).sum(axis = 1) - num_annotators)/(num_annotators * (num_annotators - 1))

# print(bigtom_P_i)

bigtom_P_bar = bigtom_P_i.mean()
bigtom_P_e_bar = (bigtom_p_j ** 2).sum()
opentom_P_bar = opentom_P_i.mean()
opentom_P_e_bar = (opentom_p_j ** 2).sum()

bigtom_kappa = (bigtom_P_bar - bigtom_P_e_bar) / (1 - bigtom_P_e_bar)
opentom_kappa = (opentom_P_bar - opentom_P_e_bar) / (1 - opentom_P_e_bar)
print('Kappa for change to what task: ', bigtom_kappa)
print('Kappa for change to what task: ', opentom_kappa)

bigtom_kappa = fleiss_kappa(bigtom_annotate_new, method='fleiss')
opentom_kappa = fleiss_kappa(opentom_annotate_new, method='fleiss')
print('Fleiss Kappa for change to what task: ', bigtom_kappa)
print('Fleiss Kappa for change to what task: ', opentom_kappa)



Kappa for change to what task:  0.7437206388975884
Kappa for change to what task:  0.7417675536179354
Fleiss Kappa for change to what task:  0.7437206388975884
Fleiss Kappa for change to what task:  0.7417675536179354


**Part 5. Finalize dataset**



1. Count number of instances for each label.

In [7]:
a = np.array([0,0,0])
b = np.array([0,0,0])
for instance in np.array(final_bigtom_new)[bigtom_right_labels]:
#for instance in np.array(final_bigtom_new)[bigtom_right_labels]:
    if instance['groundtruth_classification'] == 'hard_negative':
        a[1] += 1
    elif instance['groundtruth_classification'] == 'easy_negative':
        a[2] += 1
    elif instance['groundtruth_classification'] == 'positive':
        a[0] += 1
for instance in np.array(final_opentom_new)[opentom_right_labels]:
    if instance['groundtruth_classification'] == 'hard_negative':
        b[1] += 1
    elif instance['groundtruth_classification'] == 'easy_negative':
        b[2] += 1
    else:
        b[0] += 1
        
print(a)
print(a/a.sum())
print(b)
print(b/b.sum())

[501 246 422]
[0.42857143 0.21043627 0.3609923 ]
[511 231 504]
[0.41011236 0.18539326 0.40449438]


2. Create a filter for sampling easy negative instances during experiments. We use seed value to make sure that the results are reproducible. 

In [8]:
new_bigtom_map = {instance['ID']: idx  for idx, instance in enumerate(final_bigtom_new)}
new_opentom_map = {instance['ID']: idx  for idx, instance in enumerate(final_opentom_new)}

In [9]:
# runnable mask
# here, we do random sampling on easy negatives, just for the experiments.
# bigtom_filter, opentom_filter --> actual dataset we publish
# runnable_bigtom_filter, runnable_opentom_filter --> ones used in the experiment.
import random
random.seed(0)
np.random.seed(0)
# for fairness, we use same number of hard and easy negatives. 
bigtom_easy_negs = [i for i, instance in enumerate(np.array(final_bigtom_new)[bigtom_right_labels]) if instance['groundtruth_classification'] == 'easy_negative']
bigtom_chosen_easy_negs = np.random.choice(bigtom_easy_negs, size=246, replace=False) #np.array(random.sample(bigtom_easy_negs, 231))
# bigtom_chosen_easy_negs
runnable_bigtom_filter = np.array([instance['groundtruth_classification'] != 'easy_negative' for instance in np.array(final_bigtom_new)[bigtom_right_labels]])
# runnable_bigtom_filter
runnable_bigtom_filter[bigtom_chosen_easy_negs] = True
print(runnable_bigtom_filter)

opentom_easy_negs = [i for i, instance in enumerate(np.array(final_opentom_new)[opentom_right_labels]) if instance['groundtruth_classification'] == 'easy_negative']
opentom_chosen_easy_negs = np.random.choice(opentom_easy_negs, size=231, replace=False) #np.array(random.sample(bigtom_easy_negs, 231))
# bigtom_chosen_easy_negs
runnable_opentom_filter = np.array([instance['groundtruth_classification'] != 'easy_negative' for instance in np.array(final_opentom_new)[opentom_right_labels]])
# runnable_bigtom_filter
runnable_opentom_filter[opentom_chosen_easy_negs] = True
print(runnable_opentom_filter)


[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True False]


In [ ]:
with open('bigtom_runnable_filter.npy', 'wb') as f:
    np.save(f, runnable_bigtom_filter)
with open('opentom_runnable_filter.npy', 'wb') as f:
    np.save(f, runnable_opentom_filter)